# Import Libraries

In [19]:
import datetime
import time

import pandas as pd

from sklearn.ensemble import RandomForestRegressor

# Load Dataset

In [26]:
df_train = pd.read_csv('data/movies_train.csv')
df_test = pd.read_csv('data/movies_test.csv')

# Explore Dataset

In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [40]:
df_train.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083


In [29]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           243 non-null    object 
 1   distributor     243 non-null    object 
 2   genre           243 non-null    object 
 3   release_time    243 non-null    object 
 4   time            243 non-null    int64  
 5   screening_rat   243 non-null    object 
 6   director        243 non-null    object 
 7   dir_prev_bfnum  107 non-null    float64
 8   dir_prev_num    243 non-null    int64  
 9   num_staff       243 non-null    int64  
 10  num_actor       243 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 21.0+ KB


In [41]:
df_test.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7


### 결측치 확인

In [35]:
df_train.isnull().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [36]:
df_test.isnull().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
dtype: int64

> dir_prev_bfnum 외에는 결측치가 없어 수월하다. 다만, train 데이터의 dir_prev_bfnum은 결측치가 50%를 초과했으므로 추후 drop해야 할 것 같다.

### title
- 제목

In [39]:
df_train['title'].unique()

array(['개들의 전쟁', '내부자들', '은밀하게 위대하게', '나는 공무원이다', '불량남녀',
       '강철대오 : 구국의 철가방', '길위에서', '회사원', '1789, 바스티유의 연인들', '청춘그루브',
       'AV 아이돌', '무서운 집', '해로', '짓', '철가방 우수氏', '자유의 언덕', '써니',
       '별이 빛나는 밤', '카트', '하하하', '두근두근 내 인생', '성실한 나라의 앨리스', '원령',
       '관능의 법칙', '안녕, 투이', '영도', '이웃집 좀비', '러브앤서울 복숭아마을 도색소동기', '무법자',
       '일탈여행 : 프라이빗 아일랜드', '로맨틱 헤븐', '겨울나비', '다이노 타임', '물 없는 바다',
       '오싹한 연애', '마이 페어 웨딩', '원나잇 온리', '일대일', '하늘의 황금마차', '러브픽션',
       '불타는 내 마음', '끝과 시작', '7월 32일', '청춘학당 : 풍기문란 보쌈 야사', '부당거래',
       '콘돌은 날아간다', '페어 러브', '극적인 하룻밤', '빅매치',
       '극장판 뛰뛰빵빵 구조대 미션: 둥둥이를 구하라!', '황구', '워킹걸',
       '나의 선택 - 잊혀진 가방 그 못다한 이야기', '에스엠타운 더 스테이지', '미조', '원더풀 라디오', '막걸스',
       '미궁: 비밀애', '파괴된 사나이', '친구 2', '              개를 훔치는 완벽한 방법',
       '청춘정담', '이방인들', '간첩', '그 참을 수 없는 맛', '도리화가', '의뢰인', '열정같은소리하고있네',
       '화끈한 써비스: 어느 잔인한 미용사의', '세상끝의 사랑', '어디로 갈까요?', '완전 소중한 사랑',
       '사물의 비밀', '청포도 사탕: 17년 전의 약속', '아저씨', '원 오브 어 카인드 3D', '사랑이 무서워',
       '검은손', '덕수리 5형제

In [38]:
len(df_train['title'].unique())

600

> 영화 제목은 600개 데이터 포인트가 모두 달라서 drop해야 할 것 같다. 단, 제목에 포함된 단어만으로 그룹핑을 하거나 제목의 길이에 따른 관객 수 차이가 있는지?

### distributor
- 배급사

### genre
- 장르

### release_time
- 개봉일

### time
- 상영시간 (분)

### screening_rat
- 상영등급

### director
- 감독 이름

### dir_prev_bfnum
- 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수 (단 관객수가 알려지지 않은 영화 제외)

### dir_prev_num
- 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수 (단 관객수가 알려지지 않은 영화 제외)

### num_staff
- 스태프 수

### num_actor
- 주연 배우 수

### box_off_num
- Label, 관객 수

### 상관 분석

# Preprocessing

### Missing Values

### One Hot Encoding

# Modeling

# Prediction

In [ ]:
prediction = model.predict(test)

In [ ]:
# score

# Submission

In [8]:
submission = pd.read_csv('data/submission.csv')
submission

,title,box_off_num
0,용서는 없다,0
1,아빠가 여자를 좋아해,0
2,하모니,0
3,의형제,0
4,평행 이론,0
...,...,...
238,해에게서 소년에게,0
239,울보 권투부,0
240,어떤살인,0
241,말하지 못한 비밀,0


In [11]:
# submission['box_off_num'] = prediction
# submission

In [25]:
# timestr = time.strftime("%Y%m%d")

'20221221'

In [10]:
# submission.to_csv('.csv',index = False)